In [ ]:
import os
from glob import glob
import cv2
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from unet.model.architecture import create_unet_model

# Get image paths and inspect some images

Note that the images are the training input features and that the masks are the targets.

In [ ]:
train_image_files = glob('../data/images/*')
train_mask_files = glob('../data/masks/*')
print("Number of train files: {}".format(len(train_image_files)))
print("Number of train masks: {}".format(len(train_mask_files)))

In [ ]:
sample_image_path = train_image_files[-1]
sample_image = cv2.imread(sample_image_path)
sample_mask_path = train_mask_files[-1]
sample_mask = cv2.imread(sample_mask_path)

In [ ]:
sample_image.shape, sample_mask.shape

In [ ]:
nr_samples = 3
fig, axes = plt.subplots(nr_samples, 2, figsize=(10, 5*nr_samples))

for (ax_image, ax_mask), image_path, mask_path in zip(axes, train_image_files, train_mask_files):
    ax_image.imshow(cv2.imread(image_path)[:,:,0], cmap='seismic', interpolation='bilinear')
    ax_mask.imshow(cv2.imread(mask_path), cmap='gray', interpolation='bilinear')

# Create the model

In [ ]:
input_image = keras.layers.Input((sample_image.shape[0], sample_image.shape[1], 1), name='image')
unet_model = create_unet_model(input_image, batchnorm=True)